# Exercise 06 - Shape optimization of trusses

Shape optimization means that given a fixed topology of a truss, we want optimize its stiffness by modifying some node positions. In this particular example, we investigate the optimal shape of a railway bridge like in the photograph here:

![Bridge](../figures/bridge.jpeg)


In [ ]:
from math import sqrt

import matplotlib.pyplot as plt
import torch
from simple_truss import Truss

torch.set_default_dtype(torch.double)

So let's start by defining the base truss topology of the bridge without considering the exact shape for now. We create a simple rectangular bridge that has all the bars seen in the photo. The truss is fixed at the bottom left side and simply supported at the bottom right side. The load is distributed along the bottom edge of the bridge, which represents the train track.

In [ ]:
# Dimensions
A = 17
B = 2

# Nodes
n1 = torch.linspace(0.0, 5.0, A)
n2 = torch.linspace(0.0, 0.5, B)
n1, n2 = torch.stack(torch.meshgrid(n1, n2, indexing="xy"))
nodes = torch.stack([n1.ravel(), n2.ravel()], dim=1)

# Elements
elements = []
for i in range(A - 1):
    for j in range(B):
        elements.append([i + j * A, i + 1 + j * A])
for i in range(A):
    for j in range(B - 1):
        elements.append([i + j * A, i + A + j * A])
for i in range(A - 1):
    for j in range(B - 1):
        if i >= (A - 1) / 2:
            elements.append([i + j * A, i + 1 + A + j * A])
        else:
            elements.append([i + 1 + j * A, i + A + j * A])

# Forces at bottom edge
forces = torch.zeros_like(nodes)
forces[1 : A - 1, 1] = -0.1

# Constraints by the supports
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[0, 0] = True
constraints[0, 1] = True
constraints[A - 1, 1] = True

# Areas
areas = torch.ones((len(elements)))

# Truss
bridge = Truss(nodes.clone(), elements, forces, constraints, areas, E=500.0)
bridge.plot()

## Task 1 - Preparation of design variables

We want to restrict the shape optimization problem to the vertical displacement of the top nodes. Other nodes should not be modified - the train track should remain a flat line.

a) Create a boolean tensor `mask` of the same shape as `nodes`. It should be `True` for the vertical degree of freedom of the top nodes and `False` for every other degree of freedom. Essentially it should mask out those nodal degrees of freedom which should be optimized.

b) Create initial values `x_0` of the masked top node positions. Set limits to the deformation (`x_min`, `x_max`) such that nodes can move down by 0.4 and up by 0.5 units.

c) Compute the current volume of the truss `V0`. We will use this as a constraint in the optimization problem such that the optimized solution does not exceed this initial volume.

In [ ]:
# Mask for design variables.

# Limits on design variables

# Compute current volume

## Task 2 - Optimization

You are provided with the `box_constrained_decent` function for minimization and `MMA` function for approximation from previous exercises. Using these, you should write a function `optimize(truss, x_0, x_min, x_max, V_0, mask, iter)` that takes a `truss` and optimizes its (masked) nodal positions `x` such that it minimizes the compliance using a maximum volume `V_0`. The function should return a list $\mathbf{x}^0, \mathbf{x}^1, \mathbf{x}^2, ...$ containing the iteration steps of the optimization procedure. 

In [ ]:
def box_constrained_decent(func, x_init, x_lower, x_upper, eta=0.1, max_iter=100):
    x = x_init.clone().requires_grad_()
    for _ in range(max_iter):
        grad = torch.autograd.grad(func(x).sum(), x)[0]
        x = x - eta * grad
        x = torch.clamp(x, x_lower, x_upper)
    return x

In [ ]:
def MMA(func, x_k, L_k, U_k):
    x_lin = x_k.clone().requires_grad_()
    grads = torch.autograd.grad(func(x_lin), x_lin)[0]
    f_k = func(x_k)

    def approximation(x):
        res = f_k * torch.ones_like(x[..., 0])
        for j, grad in enumerate(grads):
            if grad < 0.0:
                p = 0
                q = -((x_k[j] - L_k[j]) ** 2) * grad
            else:
                p = (U_k[j] - x_k[j]) ** 2 * grad
                q = 0
            res -= p / (U_k[j] - x_k[j]) + q / (x_k[j] - L_k[j])
            res += p / (U_k[j] - x[..., j]) + q / (x[..., j] - L_k[j])
        return res

    return approximation, grads

In [ ]:
def optimize(truss, x_0, x_min, x_max, V_0, mask, iter):
    s = 0.7

    # Set up lists for L, U, x
    L = []
    U = []
    x = [x_0]

    # Define the initial value, lower bound, and upper bound of "mu"
    mu_0 = torch.tensor([0.01])
    mu_lower = torch.tensor([1e-10])
    mu_upper = torch.tensor([100.0])

    def f(x):
        # Update nodes

        # Solve truss with updated nodes

        # Return compliance
        pass

    def g(x):
        # Update nodes

        # Return constraint function
        pass

    for k in range(iter):
        # Update asymptotes with heuristic procedure (see Exercise 04)

        # Compute lower and upper move limit in this step

        # Compute the current approximation function and save gradients

        # Define the Lagrangian

        # Define x_star by minimizing the Lagrangian w. r. t. x analytically
        def x_star(mu):
            pass

        # Define (-1 times) the dual function
        def dual_function(mu):
            pass

        # Compute the maximum of the dual function

        # Compute current optimal point with dual solution

    return x

In [ ]:
x_opt = optimize(bridge, x_0, x_min, x_max, V0, mask, iter=50)

In [ ]:
# Plot the development of design variables
plt.plot(torch.stack(x_opt).detach())
plt.xlabel("Iteration")
plt.ylabel("Values $x_i$")
plt.show()

# Plot the optimized bridge
bridge.plot(node_labels=False)